In [193]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [194]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import regex as rex
import date_library as datelib
import utils as u

In [229]:
try:
    # df = pd.read_csv("activity_reduced_before_attributes.csv")
    df = pd.read_csv("member_d.csv")
except:
    # df = pd.read_csv("activity_reduced_before_attributes.csv.gz")
    df = pd.read_csv("member_d.csv.gz")
    

In [230]:
df.columns, df.shape

(Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
        'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
        'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size'],
       dtype='object'),
 (835129, 12))

# Create month and day of week columns for Booking date and flight date
Column names: book_mo, book_dofwk, flight_mo, flight_daywk

In [231]:
df1 = u.get_dayofweek_month(df, df_date_col='BOOKING_DATE', 
        labels=('booking_date', 'booking_mo', 'booking_dowk'))

In [232]:
df2 = u.get_dayofweek_month(df1, df_date_col='FLIGHT_DATE', 
        labels=('flight_date', 'flight_mo', 'flight_dowk'))

In [233]:
df2.columns

Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
       'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
       'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
       'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk',
       'flight_mo'],
      dtype='object')

In [234]:
dfm = pd.read_csv("MEMBERS_CATALOG.csv")
dfm.columns

Index(['MEMBER_ID', 'TIER_LEVEL', 'TIER_LEVEL_DESCRIPTION', 'PREVIOUS_TIER',
       'LAST_TIER_CHANGE_DATE', 'STATUS', 'ENROLL_DATE', 'GENDER',
       'BIRTH_DATE', 'NATIONALITY', 'ADDR_COUNTRY', 'COMMUNICATION_LANGUAGE',
       'CONNECTMILES_OFFERS_AND_NEWS', 'E_STATEMENTS',
       'COPA_AIRLINES_OFFERS_AND_NEWS', 'CCD_ACTIVE_IND', 'MILES_BALANCE',
       'TOTAL_MILES_SINCE_ENROLLMENT'],
      dtype='object')

In [235]:
dfm = dfm[['MEMBER_ID', 'TIER_LEVEL', 'GENDER', 'BIRTH_DATE']]

In [236]:
dfm1 = dfm[pd.isna(dfm['GENDER'])]

In [237]:
dfm.BIRTH_DATE

0        1962-02-14
1        1979-01-16
2        1970-12-21
3        1974-07-31
4        1988-09-21
            ...    
96840    1964-11-05
96841    1980-11-28
96842    1983-08-03
96843    1973-09-15
96844    1985-05-05
Name: BIRTH_DATE, Length: 96845, dtype: object

In [238]:
import datetime
today = datetime.date.today().strftime("%Y-%m-%d")
today1 = datelib.dateTimePTYToTimestamp(today)

In [239]:
today1 

1653577200

In [240]:
int( (today1-birthdate[0]) / (365*24*3600) )

60

# Compute passenger age when the flight departed. 

In [241]:
x = pd.merge(df2, dfm, how='inner', on='MEMBER_ID')

In [242]:
x.columns

Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
       'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
       'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
       'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk', 'flight_mo',
       'TIER_LEVEL', 'GENDER', 'BIRTH_DATE'],
      dtype='object')

In [243]:
birthdate = datelib.date_to_timestamp(x['BIRTH_DATE'])

In [244]:
x['age_at_flight'] = (x['FLIGHT_DATE'] - birthdate) / (365*24*3600)
x['age_at_flight'] = x.age_at_flight.apply(int)
# pd.isna(x['BOOKING_DATE']).sum()
#x['age_at_flight'] = x['age_at_flight'].astype(int)

In [245]:
x['age_at_flight']

0         56
1         56
2         56
3         55
4         55
          ..
835124    30
835125    36
835126    32
835127    62
835128    63
Name: age_at_flight, Length: 835129, dtype: int64

In [246]:
x.columns

Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
       'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
       'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
       'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk', 'flight_mo',
       'TIER_LEVEL', 'GENDER', 'BIRTH_DATE', 'age_at_flight'],
      dtype='object')

In [247]:
# Party size: 9 values
# Need trip duration. 

In [248]:
attribute_cols = ['MEMBER_ID', 'booking_dowk', 'booking_mo', 'flight_dowk', 
                  'flight_mo', 'age_at_flight', 'PARTY_SZ']
attributes = x[attribute_cols]

# 67 passenges are 2 years old? Not possible! Check this out. 

In [252]:
pax_age2 = attributes[attributes['age_at_flight'] == 2].head()
pax_age2

,MEMBER_ID,booking_dowk,booking_mo,flight_dowk,flight_mo,age_at_flight,PARTY_SZ
195475,244949672,3,9,5,10,2,1
195476,244949672,3,10,4,11,2,1
195477,244949672,2,9,4,10,2,1
195478,244949672,2,8,5,8,2,1
195479,244949672,5,7,3,8,2,1


In [253]:
attributes.age_at_flight.value_counts().sort_index().head(50);

In [254]:
pax_age2.booking_mo.value_counts().sort_index()
x.booking_dowk.value_counts().sort_index()

0    179516
1    145944
2    142122
3    138827
4    130237
5     53409
6     45074
Name: booking_dowk, dtype: int64

In [255]:
df[df['MEMBER_ID'] == 230831718][['MEMBER_ID', 'FLIGHT_DATE']].head(10)

,MEMBER_ID,FLIGHT_DATE
271579,230831718,1645315200
271580,230831718,1501372800
271581,230831718,1470700800
271582,230831718,1531785600
271583,230831718,1547683200
271584,230831718,1565654400
271585,230831718,1498003200
271586,230831718,1472169600
271587,230831718,1449705600
271588,230831718,1564963200


In [256]:
dfm[dfm['MEMBER_ID'] == 230831718]

,MEMBER_ID,TIER_LEVEL,GENDER,BIRTH_DATE
10436,230831718,T3,M,2013-03-03


In [257]:
# Birth date: 2013-03-03
# Flight date: 1510012800

datelib.timestampToDateTimePTY(1510012800)

('2017-11-07', '05:00')

Age: 4 years old. 

In [258]:
dfull = u.read_activity_file()
dfull[dfull['MEMBER_ID'] == 230831718][['MEMBER_ID', 'FLIGHT_DATE']].head(10)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
dfull[dfull['MEMBER_ID'] == 230831718][['MEMBER_ID', 'FLIGHT_DATE']].head(10)

In [259]:
dfm.sort_values('BIRTH_DATE', ascending=False).head(10)

,MEMBER_ID,TIER_LEVEL,GENDER,BIRTH_DATE
95185,268505626,T1,F,2019-11-22
92244,265498544,T1,NaN,2019-08-28
95491,269085786,T1,NaN,2019-08-22
92857,265995448,T1,M,2019-08-03
92109,265424727,T1,M,2019-05-19
94942,268195062,T1CC,M,2019-05-09
93840,266834074,T1,NaN,2019-04-02
94810,268010343,T1,F,2019-01-15
88036,261747712,T1,M,2019-01-13
92198,265469877,T1,NaN,2018-11-20


In [260]:
x.columns

Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
       'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
       'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
       'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk', 'flight_mo',
       'TIER_LEVEL', 'GENDER', 'BIRTH_DATE', 'age_at_flight'],
      dtype='object')

In [262]:
attributes.columns

Index(['MEMBER_ID', 'booking_dowk', 'booking_mo', 'flight_dowk', 'flight_mo',
       'age_at_flight', 'PARTY_SZ'],
      dtype='object')

In [263]:
x.to_csv("activity_reduced_with_attributes.csv", index=0)

In [264]:
attributes.columns

Index(['MEMBER_ID', 'booking_dowk', 'booking_mo', 'flight_dowk', 'flight_mo',
       'age_at_flight', 'PARTY_SZ'],
      dtype='object')

In [265]:
x.shape

(835129, 22)